__лабораторная работа N7:__ работа с временными рядами

__цель:__ изучить методы работы с временными рядами

#  загружаем данные

In [ ]:
import numpy as np
from numpy import random as rng
import pandas as pd
import matplotlib.pyplot as plt

__monthly milk production per cow in the USA__

In [ ]:
# milk_production.csv: https://drive.google.com/file/d/1AbUl7tP_J47ST7L7kc3wxENxmpBUsumS/view

url= 'https://drive.google.com/uc?id=1AbUl7tP_J47ST7L7kc3wxENxmpBUsumS'
df = pd.read_csv(url).convert_dtypes()
df['date'] = pd.to_datetime(df['Month'])
df = df.set_index('date')
df = df[['Monthly milk production (pounds per cow)']].rename(
    columns={'Monthly milk production (pounds per cow)':'values'})
df = df.asfreq('MS')

----

In [ ]:
display(df.info())
display(df.sample(3))

# анализ данных

__1. визуальная оценка данных__

In [ ]:
fig,ax = plt.subplots(figsize = (9,3))
ax.plot(df['values'], label = 'данные', color = 'steelblue')
ax.plot(df['values'].rolling(window=12).mean(), label = 'скользящее среднее (12 месяцев)', color = 'red')
ax.set_xlabel('Месяцы', )
ax.legend(title = '', loc = 'upper left', )
ax.grid()

__оценка автокорреляции__

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(df['values'])
plt.grid()
plt.show()

__оцениваем дисперсию по графику выше__   

__если ряд имеет растущую амплитуду ....__

__... то стабилизируем дисперсию - применяем преобразование Бокса-Кокса....__

In [ ]:
from scipy.special import boxcox

In [ ]:
# df['bcx'] = boxcox( df['values'],0 )

__.... иначе - пропускаем этот шаг__

In [ ]:
# df['bcx'] = df['values']

---

In [ ]:
fig,ax = plt.subplots(2,1, figsize = (7,5),sharex=True)
ax[0].plot(df['values'], label = 'данные', color = 'steelblue')
ax[0].legend(title = '', loc = 'upper left', )
ax[0].grid()

ax[1].plot(df['bcx'], label = 'boxcox', color = 'red')
ax[1].legend(title = '', loc = 'upper left', )
ax[1].set_xlabel('Месяцы', )
ax[1].grid()

## подбираем гиперпараметры модели

__оценка сезонности__

In [ ]:
# !pip install statsmodels

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decompose = seasonal_decompose(df['bcx'])
fig = decompose.plot()
plt.show()

In [ ]:
# выбираем шаг сезонности 
S = ?

__оценка стационарности ряда  (критерий Дики-Фуллера)__

In [ ]:
from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(df['bcx'])
print('p-value = ' + str(adf_test[1]))

__если оценка высокая, то ряд нестационарный__

подбираем порядок дифференцирования d,D

In [ ]:
# дифференцирование
df['dif'] = (
    df['bcx']  
    # здесь определить количество итераций дифференцирования ряда
    # .diff()
)

adf_test = adfuller(df['dif'].fillna(0.))
print('p-value = ' + str(adf_test[1]))

In [ ]:
d = ?

---

In [ ]:
# сезонное дифференцирование
df['dif_s'] = (
    df['bcx'] 
     # здесь определить количество итераций сезонного дифференцирования ряда
     #.diff(S)
)

adf_test = adfuller(df['dif_s'].fillna(0.))
print('p-value = ' + str(adf_test[1]))

In [ ]:
D = ?

---

In [ ]:
fig,ax = plt.subplots(4,1, figsize = (7,7),sharex=True)
ax[0].plot(df['values'], label = 'данные', color = 'steelblue')
ax[0].legend(title = '', loc = 'upper left', )
ax[0].grid()

ax[1].plot(df['bcx'], label = 'boxcox', color = 'red')
ax[1].legend(title = '', loc = 'upper left', )
ax[1].grid()

ax[2].plot(df['dif'], label = 'boxcox+dif', color = 'orange')
ax[2].legend(title = '', loc = 'upper left', )
ax[2].grid()

ax[3].plot(df['dif_s'], label = 'boxcox+dif_s', color = 'magenta')
ax[3].legend(title = '', loc = 'upper left', )
ax[3].grid()

__3. анализируем автокорреляции__

подбираем p,q и P,Q

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
n_lags = S+1

---

In [ ]:
fig,ax = plt.subplots(figsize=(5,3))
plot_acf(df['dif'].fillna(0.),lags=range(n_lags),ax=ax)
ax.plot([S,S],[-1,1],color='red',linestyle='dashed',alpha=.5)
ax.grid()

q - номер последнего НЕсезонного лага, при котором __автокорреляция__ значима, но меньше длинны сезона ;    

In [ ]:
q = ?

---

In [ ]:
fig,ax = plt.subplots(figsize=(5,3))
plot_pacf(df['dif'].fillna(0.),lags=range(n_lags),ax=ax)
ax.plot([S,S],[-1,1],color='red',linestyle='dashed',alpha=.5)
ax.grid()

p - номер последнего НЕсезонного лага, при котором __частичной автокорреляция__ значима, но меньше длинны сезона ;   

In [ ]:
p = ?

---

In [ ]:
n_ses_lags = 8

lags = np.arange(1*S,n_lags*S,S)
fig,ax = plt.subplots(figsize=(10,3))
plot_acf(df['dif_s'].fillna(0.),lags=lags,ax=ax)
ax.grid()

Q - номер последнего сезонного лага, при котором __автокорреляция__ значима, но меньше длинны сезона ;    

In [ ]:
Q = ?

---

In [ ]:
lags = np.arange(1*S,n_ses_lags*S,S)
fig,ax = plt.subplots(figsize=(10,3))
plot_pacf(df['dif_s'].fillna(0.),lags=lags,ax=ax)
ax.grid()

P - номер последнего сезонного лага, при котором __частичной автокорреляция__ значима, но меньше длинны сезона ;   

In [ ]:
P = ?

# разделяем данные 

In [ ]:
display( df.index.min(), df.index.max() )

In [ ]:
# разобьём данные на обучающую и тестовую выборки
train = df[:'1974-12']
test = df['1975-01':]

In [ ]:
fig,ax = plt.subplots(figsize = (9,3))
ax.plot(train['values'], color = 'blue',label='учебная выборка')
ax.plot(test['values'], color = 'red',label='тестовая выборка')
ax.set_xlabel('Месяцы', )
ax.legend(title = '', loc = 'upper left', )
ax.grid()

# обучаем модель

In [ ]:
display( 'SARIMAX ({},{},{}), ({},{},{}), {}'.format(p,d,q,P,D,Q,S) )

In [ ]:
%%time 

from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(
        train['bcx'], 
        order = (p, d, q),
        seasonal_order = (P, D, Q, S)
    ).fit(method='lbfgs')
    
    # method='cg', ) 
    # method='lbfgs',full_output=False)

In [ ]:
print(model.summary())

# тестируем

In [ ]:
start,end  = len(train), len(train)+len(test)-1

predicted = model.predict(start, end)

__если применяли преобразование Бокса-Кокса,__   
  __то выполняем обратное преобразование__

In [ ]:
from scipy.special import inv_boxcox
# predicted = inv_boxcox( predicted, 0 )

---

In [ ]:
fig,ax = plt.subplots(figsize = (9,3))
ax.plot(train['values'], color = 'blue',label='учебная выборка')
ax.plot(test['values'], color = 'red',label='тестовая выборка')
ax.plot(predicted, color = 'green',  label='прогноз')
ax.set_xlabel('Месяцы', )
ax.legend(title = '', loc = 'upper left', )
ax.grid()

__оцениваем остатки (разница между прогнозом и тестовыми данными)__

In [ ]:
start,end  = 0, len(df)

predicted = model.predict(start, end)

__если применяли преобразование Бокса-Кокса,__   
  __то выполняем обратное преобразование__

In [ ]:
from scipy.special import inv_boxcox
# predicted = inv_boxcox( predicted, 0 )

In [ ]:
# считаем остатки
err = df['values'] - predicted 

In [ ]:
n_skip = 12*4

fig,ax = plt.subplots(2,1,figsize = (9,6),sharex=True)
ax[0].plot(train[n_skip:]['values'], color = 'blue',label='учебная выборка')
ax[0].plot(test['values'], color = 'orange',label='тестовая выборка')
ax[0].plot(predicted[n_skip:], color = 'red',label='прогноз')
ax[0].grid()
ax[0].legend()
ax[1].plot(err[n_skip:], color = 'orange',label='остаток')
ax[1].grid()
ax[1].legend()

__оценка автокореляции остатков__

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(err.fillna(0.))
plt.grid()
plt.show()

# строим прогноз

In [ ]:
forecast_periods = S*4 # на 4 сезона вперед
forecast = model.get_forecast(steps=forecast_periods)
forecast_mean = forecast.predicted_mean # значение прогноза
forecast_ci = forecast.conf_int() # предсказательный интервал

In [ ]:
fig,ax = plt.subplots(figsize=(9,5))
ax.plot(train['values'], label='данные')
ax.plot(forecast_mean, label='прогноз', color='red')
ax.fill_between(
        forecast_ci.index, 
        forecast_ci.iloc[:, 0], 
        forecast_ci.iloc[:, 1], 
        color='pink', 
        label='предсказательный интервал'
    )
ax.set_xlabel("Date")
ax.legend(loc='upper left')
ax.grid()